In [6]:
import pandas as pd
%matplotlib widget
%precision 30
import copy
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from scipy import cluster
from scipy.integrate import cumtrapz,simps
import scipy.interpolate
from scipy.interpolate import UnivariateSpline

from Funciones_auxiliares import plot_branch,get_cluster_indexs,plots_hierarchichal_clustering
plt.close('all')

In [7]:
data = '//home/lorenzo/Principal/Noise-Neuron discriminator/Datos/PCCIBNMZc'
data = pd.read_pickle(data)
#df = data[data.PatientExperiment.str.contains('HEC032|HEC031|HEC030')]
#df.reset_index(inplace = True)
Neurons = data[data.bNoise == 0]
Neurons.reset_index(inplace = True)

In [8]:
'''Ejemplo de media neuronal'''
neuron = Neurons.loc[32]
plt.plot(neuron.Mean)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
df = Neurons.copy()
fig,axes,fl = plots_hierarchichal_clustering(df)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [22]:
Cluster_selected = get_cluster_indexs(5,fl)
Emb = Neurons.loc[Cluster_selected]
x = np.concatenate(Emb.Mean.values)
x = x[0:64] #cada senial neuronal tiene 64 puntos.
x = (x - np.mean(x))/np.std(x)
x = x - x[0]

In [11]:

t = np.linspace(0,len(x) - 1.0001,10000)
t2 = np.arange(len(x))
x_interp = UnivariateSpline(t2,x,s = 0)
x_punto_interp = x_interp.derivative()
x_puntopunto_interp = x_interp.derivative(2)

x_interp = x_interp(t)
x_punto_interp = x_punto_interp(t)
x_puntopunto_interp = x_puntopunto_interp(t)


x_punto = np.gradient(x)
x_puntopunto = np.gradient(x_punto)


x_punto_integrated = cumtrapz(x_puntopunto)
x_integrated = cumtrapz(x_punto_integrated)

x_punto_integrated_interp= cumtrapz(x_puntopunto_interp)
x_integrated_interp= cumtrapz(x_punto_integrated_interp)

fig = plt.figure()
axes = fig.subplots(2,2)
axes[0][0].scatter(t2,x,label = 'senial original')
axes[0][0].plot(x_integrated,color = 'r',label = 'senial integrada')
axes[0][1].scatter(t,x_interp,color = 'g', marker = '.',label = 'senial interpolada')
axes[0][1].plot(t[1:-1],x_integrated_interp,color = 'c',label = 'senial interpolada integrada')

axes[1][0].plot(x_punto,label = 'x_punto')
axes[1][0].plot(x_puntopunto,label = 'x_puntopunto')
axes[1][1].plot(t,x_punto_interp,label = 'x_punto interp')
axes[1][1].plot(t,x_puntopunto_interp,label = 'x_puntopunto interp')
axes[0][0].legend()
axes[0][1].legend()
axes[1][0].legend()
axes[1][1].legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
fig = plt.figure()
#ax = fig.gca(projection = '3d')
#ax.plot(x_interp,x_punto_interp)
#ax.set_xlabel('x')
#ax.set_y
#label('$\dot{x}$')
plt.plot(x_interp,x_punto_interp)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
Neurons.head()

,index,PatientExperiment,Channel,Cluster,ISI,Bulk,bNoise,Mean,ZeroCrossings
0,3,HEC016\016context1\016context1,1,2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[-9.910404058695178, -10.706527972656263, -13...",0,"[5.549115009633331, 6.0470913803953605, 5.9916...",2
1,12,HEC016\016context1\016context1,2,2,"[0, 0, 1, 0, 0, 2, 0, 1, 1, 3, 2, 1, 3, 1, 2, ...","[[13.198738217761697, 11.760198708498171, 8.78...",0,"[13.243695374870788, 13.079745522544341, 12.71...",2
2,34,HEC017\017context2\017context2_prescr,113,2,"[0, 3, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, ...","[[18.892933610485304, 15.485716617075768, 8.32...",0,"[6.993260836407021, 6.865110211253, 6.51757150...",2
3,35,HEC017\017context2\017context2_prescr,113,3,"[0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...","[[8.607666061103055, 4.40269092990618, 2.82740...",0,"[2.722673147546738, 2.3590640477204063, 2.0802...",6
4,47,HEC017\017context2\017context2_prescr,118,3,"[0, 4, 1, 0, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[[5.702971353263122, 5.288244296823778, 3.4659...",0,"[4.340277642514026, 4.058565844662801, 3.92196...",2


# Sindy

Definimos Theta

In [9]:
x = x_interp
x_punto = x_punto_interp
x_puntopunto = x_puntopunto_interp
degree = 10
idx = np.arange(1,degree,2)
Theta = np.zeros([x.shape[0],degree])
Theta[:,0] = np.ones(x.shape[0])
Theta[:,1] = x
Theta[:,2] = x_punto
Theta[:,3] = x*x_punto
Theta[:,4] = x_punto**2
Theta[:,5] = x**3
Theta[:,6] = x**2*x_punto
Theta[:,7] = x**2
Theta[:,8] = x_punto**3
Theta[:,9] = x**4

Lasso

In [10]:
from sklearn.linear_model import Lasso
from scipy.integrate import odeint

classifier = Lasso(alpha=0.000000000001,max_iter=1000000)
classifier.fit(Theta,x_puntopunto)
print(Theta.shape,len(x_puntopunto))

(10000, 10) 10000


Vector de coeficientes ($\eta$)

In [12]:
coef_vector = copy.copy(classifier.coef_)
print(coef_vector)

[ 0.                    0.042579721327655966  0.29132122218515655
  0.20846415560777062  -0.13324995000322043  -0.058470565091640135
 -0.01316826256110804  -0.06451597101228136  -0.11427323629077762
 -0.005170770099244098]


Ecuaciones diferenciales

In [13]:
def f(X,t,coef_vector):
    r,rpunto = X
    u = np.array([1,r,rpunto,r*rpunto,rpunto**2,r**3,rpunto*r**2,r**2,rpunto**3,r**4])
    
    drdt = rpunto
    drpunto_dt = np.dot(coef_vector,u)
    return drdt,drpunto_dt

Integramos las ecuaciones y graficamos

In [14]:
X0 = (x[0],x_punto[0])
print('Initial conditions: X = ',X0[0],' X_punto = ',X0[1])
t = np.linspace(0,64,100000)
t2 = np.linspace(0,64,len(x))
tray = odeint(f,X0,t,args = (coef_vector,)).T

fig = plt.figure()

plt.plot(t,tray[0])
plt.plot(t2,x)


Initial conditions: X =  1.3447872870528276e-20  X_punto =  0.0009942504213135525


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
t = np.linspace(0,64,100000)
t2 = np.linspace(64,0,1000)

X0 = (0,.010)
tray = odeint(f,X0,t,args = (coef_vector,)).T

X1 = (-1,.01)
tray1 = odeint(f,X1,t,args = (coef_vector,)).T

X2 = (.1,.01)
tray2 = odeint(f,X2,t,args = (coef_vector,)).T

X3 = (.2,.01)
tray3 = odeint(f,X3,t,args = (coef_vector,)).T

X4 = 0.,.01
tray4 = odeint(f,X4,t2,args = (coef_vector,)).T

plt.figure()
plt.plot(tray[0],tray[1])
plt.plot(tray1[0],tray1[1])
plt.plot(tray2[0],tray2[1])
plt.plot(tray3[0],tray3[1])
plt.plot(tray4[0],tray4[1])

a = tray4[:,-1]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
coef_vector = copy.copy(classifier.coef_)
#coef_vector[2] = .28
#coef_vector[1] = 0.043
#coef_vector[3] = .235
#coef_vector[8] = -.1
t = np.linspace(0,100,10000)
cross = (-0.03,0.056)
ic = [cross + 0.00*(np.random.random(2) - 1/2) for i in range(1)]
plt.close('all')
plt.figure(90,figsize = (10,10))
for i in ic:
    tray = odeint(f,i,t,args = (coef_vector,)).T
    plt.plot(tray[0],tray[1])
plt.plot(x,x_punto,'red')
plt.scatter(cross[0],cross[1],color = 'red')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
cross = (-0.03, 0.056)
t = np.linspace(0,100,10000)
plt.figure(figsize = (10,10))
#Change mu1
mu1_list = np.linspace(0.055283496093749995,0.055283544921875,3,dtype = 'float128')
for mu1 in mu1_list:
    coef_vector[1] = mu1
    tray = odeint(f,cross,t,args = (coef_vector,)).T
    plt.plot(t,tray[0],label = '$\mu$ = {}'.format(mu1))
    print(mu1)
t2 = np.linspace(0,64,10000)
plt.plot(t2,x)
plt.legend()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0.055283496093749995293
0.055283520507812496103
0.055283544921874996914


In [18]:
def mov_parameter(function,n,initial_parameters,initial_condition,coef_vector):
        t = np.linspace(0,100,10000)
        mu_min,mu_max = initial_parameters
      
        up_down = []
                 
        threshold_inf = -1
        threshold_sup = .5
        for mu in np.linspace(mu_min,mu_max,5,dtype = 'float128'): 
            inf = 0
            sup = 0
            coef_vector[n] = mu
            tray = odeint(function,initial_condition,t,(coef_vector,)).T
            s = np.where(t > 22)[0]
            r = tray[0][s] #I don't care about the begining
            sup = np.where(r>threshold_sup)[0] #Find when r is higher than sup_threshold (if it is possible)
            
            if not len(sup) == 0:#maybe r just go down, never surpases threshold_sup and sup is empty
                sup = sup[0]
            else:
                sup = 100*10000
                
            inf = np.where(r<threshold_inf)[0] #Find when r is lower than threshold_inf (if it is possible)
            if not len(inf)== 0:
                inf = inf[0]
            else:
                inf = 100*10000
            print(mu,inf,sup)
            
            if sup<inf:
                a = 'up' 
                print(mu,a)
            elif inf<sup:
                a = 'down'
                print(mu,a)
            else:
                a = 'stable'    
                print(mu,a)

            if (a == 'up' and mu > mu_min): mu_min = mu
            if (a == 'down' and mu < mu_max): mu_max = mu
            if a == 'stable': continue
        plt.legend()
        return mu_min,mu_max
    
plt.figure()
mu_min = 0.055283496093749995
mu_max = 0.055283544921875
for i in range(40):        
    mu_min,mu_max = mov_parameter(f,1,[mu_min,mu_max],cross,coef_vector)
    print('mu_min = ',mu_min,'mu_max = ',mu_max)
            
t = np.linspace(0,64,1000)

coef_vector[1] = mu_max/2 + mu_min/2
print(coef_vector[1])
coef_vector[1] = 0.055283522241596389284
tray = odeint(f,cross,t,args = (coef_vector,)).T
plt.plot(t,tray[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.


0.055283496093749995293 3900 3159
0.055283496093749995293 up
0.05528350830078124223 4049 3307
0.05528350830078124223 up
0.055283520507812496103 4543 3799
0.055283520507812496103 up
0.055283532714843749978 3546 1000000
0.055283532714843749978 down
0.055283544921874996914 3363 1000000
0.055283544921874996914 down
mu_min =  0.055283520507812496103 mu_max =  0.055283532714843749978
0.055283520507812496103 4543 3799
0.055283520507812496103 up
0.055283523559570309572 4037 1000000
0.055283523559570309572 down
0.05528352661132812304 3753 1000000
0.05528352661132812304 down
0.05528352966308593651 3628 1000000
0.05528352966308593651 down
0.055283532714843749978 3546 1000000
0.055283532714843749978 down
mu_min =  0.055283520507812496103 mu_max =  0.055283523559570309572
0.055283520507812496103 4543 3799
0.055283520507812496103 up
0.05528352127075194947 4680 3936
0.05528352127075194947 up
0.055283522033691402837 5045 4300
0.055283522033691402837 up
0.055283522796630856205 4242 1000000
0.0552835227

No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.


0.05528352222442626618 5636 4890
0.05528352222442626618 up
0.05528352241516112952 4517 1000000
0.05528352241516112952 down
0.055283522605895992863 4341 1000000
0.055283522605895992863 down
0.055283522796630856205 4242 1000000
0.055283522796630856205 down
mu_min =  0.05528352222442626618 mu_max =  0.05528352241516112952
0.05528352222442626618 5636 4890
0.05528352222442626618 up
0.055283522272109982015 4928 1000000
0.055283522272109982015 down
0.05528352231979369785 4706 1000000
0.05528352231979369785 down
0.055283522367477413686 4593 1000000
0.055283522367477413686 down
0.05528352241516112952 4517 1000000
0.05528352241516112952 down
mu_min =  0.05528352222442626618 mu_max =  0.055283522272109982015
0.05528352222442626618 5636 4890
0.05528352222442626618 up
0.055283522236347195138 5917 5171
0.055283522236347195138 up
0.055283522248268124097 5288 1000000
0.055283522248268124097 down
0.055283522260189053056 5046 1000000
0.055283522260189053056 down
0.055283522272109982015 4928 1000000
0.05

...

0.055283522241596389284 1000000 1000000
0.055283522241596389284 stable
0.05528352224159643476 1000000 1000000
0.05528352224159643476 stable
0.05528352224159648023 1000000 1000000
0.05528352224159648023 stable
0.055283522241596525707 7693 1000000
0.055283522241596525707 down
mu_min =  0.05528352224159634381 mu_max =  0.055283522241596525707
0.05528352224159634381 1000000 7593
0.05528352224159634381 up
0.055283522241596389284 1000000 1000000
0.055283522241596389284 stable
0.05528352224159643476 1000000 1000000
0.05528352224159643476 stable
0.05528352224159648023 1000000 1000000
0.05528352224159648023 stable
0.055283522241596525707 7693 1000000
0.055283522241596525707 down
mu_min =  0.05528352224159634381 mu_max =  0.055283522241596525707
0.05528352224159634381 1000000 7593
0.05528352224159634381 up
0.055283522241596389284 1000000 1000000
0.055283522241596389284 stable
0.05528352224159643476 1000000 1000000
0.05528352224159643476 stable
0.05528352224159648023 1000000 1000000
0.05528352224

No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.


0.05528352224159634381 1000000 7593
0.05528352224159634381 up
0.055283522241596389284 1000000 1000000
0.055283522241596389284 stable
0.05528352224159643476 1000000 1000000
0.05528352224159643476 stable
0.05528352224159648023 1000000 1000000
0.05528352224159648023 stable
0.055283522241596525707 7693 1000000
0.055283522241596525707 down
mu_min =  0.05528352224159634381 mu_max =  0.055283522241596525707
0.05528352224159634381 1000000 7593
0.05528352224159634381 up
0.055283522241596389284 1000000 1000000
0.055283522241596389284 stable
0.05528352224159643476 1000000 1000000
0.05528352224159643476 stable
0.05528352224159648023 1000000 1000000
0.05528352224159648023 stable
0.055283522241596525707 7693 1000000
0.055283522241596525707 down
mu_min =  0.05528352224159634381 mu_max =  0.055283522241596525707
0.05528352224159634381 1000000 7593
0.05528352224159634381 up
0.055283522241596389284 1000000 1000000
0.055283522241596389284 stable
0.05528352224159643476 1000000 1000000
0.055283522241596434

No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.


0.05528352224159634381 1000000 7593
0.05528352224159634381 up
0.055283522241596389284 1000000 1000000
0.055283522241596389284 stable
0.05528352224159643476 1000000 1000000
0.05528352224159643476 stable
0.05528352224159648023 1000000 1000000
0.05528352224159648023 stable
0.055283522241596525707 7693 1000000
0.055283522241596525707 down
mu_min =  0.05528352224159634381 mu_max =  0.055283522241596525707
0.05528352224159634381 1000000 7593
0.05528352224159634381 up
0.055283522241596389284 1000000 1000000
0.055283522241596389284 stable
0.05528352224159643476 1000000 1000000
0.05528352224159643476 stable
0.05528352224159648023 1000000 1000000
0.05528352224159648023 stable
0.055283522241596525707 7693 1000000
0.055283522241596525707 down
mu_min =  0.05528352224159634381 mu_max =  0.055283522241596525707
0.05528352224159634381 1000000 7593
0.05528352224159634381 up
0.055283522241596389284 1000000 1000000
0.055283522241596389284 stable
0.05528352224159643476 1000000 1000000
0.055283522241596434

In [19]:
def mov_parameter(function,n,initial_parameters,initial_condition,coef_vector):
        max_t = 100
        t = np.linspace(0,max_t,10000)
        mu_min,mu_max = initial_parameters
      
        up_down = []
                 
        threshold_inf = -1
        threshold_sup = 1
        plt.figure()
        for mu in np.linspace(mu_min,mu_max,10,dtype = 'float128'): 
            coef_vector[n] = mu
            tray = odeint(function,initial_condition,t,(coef_vector,)).T
            s = np.where(t > 22)[0]
            r = tray[0][s] #I don't care about the beggining
            plt.plot(s,r,label = str(mu))
            sup = np.where(r>threshold_sup)[0] #Find when r is higher than sup_threshold (if it is possible)
            
            if not len(sup) == 0:#maybe r just go down, never surpases threshold_sup and sup is empty
                sup = sup[0]
            else:
                sup = max_t*100
                
            inf = np.where(r<threshold_inf)[0] #Find when r is lower than threshold_inf (if it is possible)
            if not len(inf)== 0:
                inf = inf[0]
            else:
                inf = max_t*100
            
            if sup<inf:
                a = 'up' #if firt
                print(mu,a)
            elif inf<sup:
                a = 'down'
                print(mu,a)
            else:
                a = 'stable'    
                print(mu,a)

 #           if (a == 'up' and mu > mu_min): mu_min = mu
#            if (a == 'down' and mu < mu_max): mu_max = mu
 #           if a == 'stable': continue
                
            if (a == 'down' and mu > mu_min): mu_min = mu
            if (a == 'up' and mu < mu_max): mu_max = mu
            if a == 'stable': continue
        plt.legend()
        return mu_min,mu_max
    
coef_vector = classifier.coef_
coef_vector[1] = 0.055283522241596389284

plt.figure()
mu_min = 0.28
mu_max = 0.3
for i in range(50):        
    mu_min,mu_max = mov_parameter(f,2,[mu_min,mu_max],cross,coef_vector)
    print(mu_min,mu_max)
            
t = np.linspace(0,65,1000)

coef_vector[2] = mu_max/2 + mu_min/2
print(coef_vector[2])
tray = odeint(f,cross,t,args = (coef_vector,)).T
plt.plot(t,tray[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0.28000000000000002665 down
0.28222222222222226318 down
0.2844444444444444442 down
0.28666666666666668073 down
0.28888888888888891726 down
0.29111111111111109828 down
0.29333333333333333481 up
0.29555555555555557135 up
0.29777777777777775237 up
0.2999999999999999889 up
0.29111111111111109828 0.29333333333333333481


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0.29111111111111109828 down
0.29135802469135801344 up
0.29160493827160492863 up
0.29185185185185184378 up
0.29209876543209875897 up
0.29234567901234567413 up
0.2925925925925925893 up
0.29283950617283950447 up
0.29308641975308641966 up
0.29333333333333333481 up
0.29111111111111109828 0.29135802469135801344


...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0.2913212221851565157 down
0.29132122218515657327 stable
0.29132122218515663084 stable
0.29132122218515668838 stable
0.29132122218515674596 stable
0.29132122218515680353 stable
0.2913212221851568611 stable
0.29132122218515691864 stable
0.2913212221851569762 stable
0.29132122218515703378 up
0.2913212221851565157 0.29132122218515703378


/home/lorenzo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  # Remove the CWD from sys.path while we load stuff.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0.2913212221851565157 down
0.29132122218515657327 stable
0.29132122218515663084 stable
0.29132122218515668838 stable
0.29132122218515674596 stable
0.29132122218515680353 stable
0.2913212221851568611 stable
0.29132122218515691864 stable
0.2913212221851569762 stable
0.29132122218515703378 up
0.2913212221851565157 0.29132122218515703378
0.29132122218515677


In [20]:
t = np.linspace(0,100,10000)

plt.figure()
coef_vector[1] = 0.055283522241596389284
coef_vector[2] = 0.2913212221851568611
tray = odeint(f,cross,t,args = (coef_vector,)).T
plt.plot(t,tray[0])
plt.figure()


/home/lorenzo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  This is separate from the ipykernel package so we can avoid doing imports until


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/lorenzo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<Figure size 640x480 with 0 Axes>

In [21]:
cross = (-0.03, 0.056)
t = np.linspace(0,100,10000)
print(coef_vector)
print(mu2_list)
plt.figure(figsize = (10,10))
#Change mu1
mu2_list = np.linspace(0.291320000,  0.29132803125000006,3)
for mu2 in mu2_list:
    coef_vector[2] = mu2
    tray = odeint(f,cross,t,args = (coef_vector,)).T
    plt.plot(t,tray[0],label = '$\mu$ = {}'.format(mu2))

t2 = np.linspace(0,64,10000)
plt.plot(t2,x)
plt.legend()

[ 0.                    0.05528352224159639   0.2913212221851569
  0.20846415560777062  -0.13324995000322043  -0.058470565091640135
 -0.01316826256110804  -0.06451597101228136  -0.11427323629077762
 -0.005170770099244098]


NameError: name 'mu2_list' is not defined

In [22]:
t = np.linspace(0,64,100000)
t2 = np.linspace(64,0,1000)

X0 = (0,.010)
tray = odeint(f,X0,t,args = (coef_vector,)).T

X1 = (-1,.01)
tray1 = odeint(f,X1,t,args = (coef_vector,)).T

X2 = (.1,.01)
tray2 = odeint(f,X2,t,args = (coef_vector,)).T

X3 = (.2,.01)
tray3 = odeint(f,X3,t,args = (coef_vector,)).T

X4 = 0.,.01
tray4 = odeint(f,X4,t2,args = (coef_vector,)).T

plt.figure()
plt.plot(tray[0],tray[1])
plt.plot(tray1[0],tray1[1])
plt.plot(tray2[0],tray2[1])
plt.plot(tray3[0],tray3[1])
plt.plot(tray4[0],tray4[1])

/home/lorenzo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [207]:
a = tray4[0][-1],tray4[1][-1]
x = np.concatenate([tray4[0],tray[0]],axis = 0)
y = np.concatenate([tray4[1],tray[1]])

plt.figure()
#plt.plot(x,x_punto)
plt.plot(x,y)

/home/lorenzo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [185]:
x = x_interp
x_punto = x_punto_interp
x_puntopunto = x_puntopunto_interp
degree = 7
idx = np.arange(1,degree,2)
Theta = np.zeros([x.shape[0],degree])
Theta[:,0] = np.ones(x.shape[0])
Theta[:,1] = x
Theta[:,2] = x_punto
Theta[:,3] = x*x_punto
Theta[:,4] = x_punto**2
Theta[:,5] = x**3
Theta[:,6] = x**2*x_punto

In [19]:
classifier = Lasso(alpha=0.000000001,max_iter=10000000)
classifier.fit(Theta,x_puntopunto)
print(Theta.shape,len(x_puntopunto))

(10000, 7) 10000


In [20]:
coef_vector = classifier.coef_
print(coef_vector)

[ 0.         -0.09843611 -0.11180471  0.29717996 -0.18435317 -0.01180283
  0.04806139]


In [21]:
def f(X,t):
    r,rpunto = X
    u = np.array([1,r,rpunto,r*rpunto,rpunto**2,r**3,rpunto*r**2])
    
    drdt = rpunto
    drpunto_dt = np.dot(coef_vector,u)
    return drdt,drpunto_dt

In [22]:
X0 = (x[0],x_punto[0])
print('Initial conditions: X = ',X0[0],' X_punto = ',X0[1])
t = np.linspace(0,64,100000)
t2 = np.linspace(0,64,len(x))
tray = odeint(f,X0,t).T

fig = plt.figure()

plt.plot(t,tray[0])
plt.plot(t2,x)


Initial conditions: X =  1.3447872870528276e-20  X_punto =  0.0009942504213135525


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
t = np.linspace(0,64,10000)
t2 = np.linspace(90,0,10000)
X0 = (-1.1,.010)
tray = odeint(f,X0,t).T

X1 = (-1,.01)
tray1 = odeint(f,X1,t).T

X2 = (-.9,.01)
tray2 = odeint(f,X2,t).T

X3 = (-2,-1)
tray3 = odeint(f,X3,t).T

x4 = tray[:,-1]
tray4 = odeint(f,x4,t2).T


plt.figure()
plt.plot(tray[0],tray[1])
plt.plot(tray1[0],tray1[1])
plt.plot(tray2[0],tray2[1])
plt.plot(tray3[0],tray3[1])
plt.plot(tray4[0],tray4[1])

plt.plot(x,x_punto)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Probemos con otra neurona

In [24]:
Cluster_selected = get_cluster_indexs(10,fl)
Emb = Neurons.loc[Cluster_selected]
x = np.concatenate(Emb.Mean.values)
x = x[1:64]
x = (x - np.mean(x))/np.std(x)
x = x - x[0]


In [25]:

t = np.linspace(0,len(x),100000)
t2 = np.arange(len(x))
x_interp = UnivariateSpline(t2,x,s = 0)
x_punto_interp = x_interp.derivative()
x_puntopunto_interp = x_interp.derivative(2)

x_interp = x_interp(t)
x_punto_interp = x_punto_interp(t)
x_puntopunto_interp = x_puntopunto_interp(t)


x_punto = np.gradient(x)
x_puntopunto = np.gradient(x_punto)


x_punto_integrated = cumtrapz(x_puntopunto)
x_integrated = cumtrapz(x_punto_integrated)

x_punto_integrated_interp= cumtrapz(x_puntopunto_interp)
x_integrated_interp= cumtrapz(x_punto_integrated_interp)

fig = plt.figure()
axes = fig.subplots(2,2)
axes[0][0].scatter(t2,x,label = 'senial original')
axes[0][0].plot(x_integrated,color = 'r',label = 'senial integrada')
axes[0][1].scatter(t,x_interp,color = 'g', marker = '.',label = 'senial interpolada')
axes[0][1].plot(t[1:-1],x_integrated_interp,color = 'c',label = 'senial interpolada integrada')

axes[1][0].plot(x_punto,label = 'x_punto')
axes[1][0].plot(x_puntopunto,label = 'x_puntopunto')
axes[1][1].plot(t,x_punto_interp,label = 'x_punto interp')
axes[1][1].plot(t,x_puntopunto_interp,label = 'x_puntopunto interp')
axes[0][0].legend()
axes[0][1].legend()
axes[1][0].legend()
axes[1][1].legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [26]:
x = x_interp
x_punto = x_punto_interp
x_puntopunto = x_puntopunto_interp
degree = 10
idx = np.arange(1,degree,2)
Theta = np.zeros([x.shape[0],degree])
Theta[:,0] = np.ones(x.shape[0])
Theta[:,1] = x
Theta[:,2] = x_punto
Theta[:,3] = x*x_punto
Theta[:,4] = x_punto**2
Theta[:,5] = x**3
Theta[:,6] = x**2*x_punto
Theta[:,7] = x**2
Theta[:,8] = x_punto**3
Theta[:,9] = x**4

Lasso

In [27]:
from sklearn.linear_model import Lasso
from scipy.integrate import odeint
classifier = Lasso(alpha=0.000001,max_iter=1000000)
classifier.fit(Theta,x_puntopunto)
print(Theta.shape,len(x_puntopunto))

(100000, 10) 100000


Vector de coeficientes ($\eta$)

In [28]:
coef_vector = classifier.coef_
print(coef_vector)

[ 0.          0.05155227  0.44021962  0.07512939 -0.43876236 -0.04946516
 -0.03408778 -0.0546355  -0.81142663 -0.00512404]


Ecuaciones diferenciales

In [29]:
def f(X,t):
    r,rpunto = X
    u = np.array([1,r,rpunto,r*rpunto,rpunto**2,r**3,rpunto*r**2,r**2,rpunto**3,r**4])
    
    drdt = rpunto
    drpunto_dt = np.dot(coef_vector,u)
    return drdt,drpunto_dt

Integramos las ecuaciones y graficamos

In [30]:
X0 = (x[0],x_punto[0])
print('Initial conditions: X = ',X0[0],' X_punto = ',X0[1])
t = np.linspace(0,64,100000)
t2 = np.linspace(0,64,len(x))
tray = odeint(f,X0,t).T

fig = plt.figure()

plt.plot(t,tray[0])
plt.plot(t2,x)


Initial conditions: X =  -2.689574574105655e-19  X_punto =  -0.04288473923798376


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [31]:
t = np.linspace(0,64,1000)
t2 = np.linspace(64,0,1000)

t = t2
X0 = (x[0],x_punto[0])
tray = odeint(f,X0,t).T

X1 = (-.1,.01)
tray1 = odeint(f,X1,t).T

X2 = (1,.01)
tray2 = odeint(f,X2,t).T

X3 = (-.5,.01)
tray3 = odeint(f,X3,t).T



plt.figure()
plt.plot(tray[0],tray[1])
plt.plot(tray1[0],tray1[1])
plt.plot(tray2[0],tray2[1])
plt.plot(tray3[0],tray3[1])

a = tray[:,-1]
a2 = tray3[:,-1]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [32]:
t = np.linspace(0,1000,10000)
ic = [a + 0.4*(np.random.random(2) - 1/2) for i in range(10)]
plt.close('all')
plt.figure(90)
for i in ic:
    tray = odeint(f,i,t).T
    plt.plot(tray[0],tray[1])
    
t = np.linspace(0,1000,10000)
ic = [a2 +.5*(np.random.random(2) - 1/2) for i in range(10)]
for i in ic:
    tray = odeint(f,i,t).T
    plt.plot(tray[0],tray[1])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

EL foco es un foco repulsor

In [33]:
t = np.linspace(0,100,100000)
ic = [a + 0.4*(np.random.random(2) - 1/2) for i in range(1)]
plt.close('all')
plt.figure(90)
for i in ic:
    tray = odeint(f,i,t).T
    plt.plot(t,tray[0])

ic = [a2 +1*(np.random.random(2) - 1/2) for i in range(1)]
for i in ic:
    tray = odeint(f,i,t).T
    plt.plot(t,tray[0])
plt.plot(t,x)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Una neurona mas

In [34]:
Cluster_selected = get_cluster_indexs(24,fl)
Emb = Neurons.loc[Cluster_selected]
x = np.concatenate(Emb.Mean.values)
x = x[1:64]
x = (x - np.mean(x))/np.std(x)
x = x - x[0]


In [35]:

t = np.linspace(0,len(x),100000)
t2 = np.arange(len(x))
x_interp = UnivariateSpline(t2,x,s = 0)
x_punto_interp = x_interp.derivative()
x_puntopunto_interp = x_interp.derivative(2)

x_interp = x_interp(t)
x_punto_interp = x_punto_interp(t)
x_puntopunto_interp = x_puntopunto_interp(t)


x_punto = np.gradient(x)
x_puntopunto = np.gradient(x_punto)


x_punto_integrated = cumtrapz(x_puntopunto)
x_integrated = cumtrapz(x_punto_integrated)

x_punto_integrated_interp= cumtrapz(x_puntopunto_interp)
x_integrated_interp= cumtrapz(x_punto_integrated_interp)

fig = plt.figure()
axes = fig.subplots(2,2)
axes[0][0].scatter(t2,x,label = 'senial original')
axes[0][0].plot(x_integrated,color = 'r',label = 'senial integrada')
axes[0][1].scatter(t,x_interp,color = 'g', marker = '.',label = 'senial interpolada')
axes[0][1].plot(t[1:-1],x_integrated_interp,color = 'c',label = 'senial interpolada integrada')

axes[1][0].plot(x_punto,label = 'x_punto')
axes[1][0].plot(x_puntopunto,label = 'x_puntopunto')
axes[1][1].plot(t,x_punto_interp,label = 'x_punto interp')
axes[1][1].plot(t,x_puntopunto_interp,label = 'x_puntopunto interp')
axes[0][0].legend()
axes[0][1].legend()
axes[1][0].legend()
axes[1][1].legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [36]:
x = x_interp
x_punto = x_punto_interp
x_puntopunto = x_puntopunto_interp
degree = 10
idx = np.arange(1,degree,2)
Theta = np.zeros([x.shape[0],degree])
Theta[:,0] = np.ones(x.shape[0])
Theta[:,1] = x
Theta[:,2] = x_punto
Theta[:,3] = x*x_punto
Theta[:,4] = x_punto**2
Theta[:,5] = x**3
Theta[:,6] = x**2*x_punto
Theta[:,7] = x**2
Theta[:,8] = x_punto**3
Theta[:,9] = x**4

Lasso

In [37]:
from sklearn.linear_model import Lasso
from scipy.integrate import odeint
classifier = Lasso(alpha=0.0000001,max_iter=100000)
classifier.fit(Theta,x_puntopunto)
print(Theta.shape,len(x_puntopunto))

(100000, 10) 100000


/home/lorenzo/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 21.04400919268219, tolerance: 0.13212147197042645
  positive)


Vector de coeficientes ($\eta$)

In [38]:
coef_vector = classifier.coef_
print(coef_vector)

[ 0.         -0.05209535  0.34361056  0.28813098 -0.58926565 -0.10090937
  0.04591659 -0.16573967 -0.2793489  -0.01342172]


Ecuaciones diferenciales

In [39]:
def f(X,t):
    r,rpunto = X
    u = np.array([1,r,rpunto,r*rpunto,rpunto**2,r**3,rpunto*r**2,r**2,rpunto**3,r**4])
    
    drdt = rpunto
    drpunto_dt = np.dot(coef_vector,u)
    return drdt,drpunto_dt

Integramos las ecuaciones y graficamos

In [40]:
X0 = (-.001, 0.000003)
print('Initial conditions: X = ',X0[0],' X_punto = ',X0[1])
t = np.linspace(0,64,10000)
t2 = np.linspace(0,64,len(x))
tray = odeint(f,X0,t).T

fig = plt.figure()

plt.plot(t,tray[0])
plt.plot(t2,x)


Initial conditions: X =  -0.001  X_punto =  3e-06


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [41]:
t = np.linspace(0,1,1000)
t2 = np.linspace(150,0,1000)


X0 = (x[0],x_punto[0])
tray = odeint(f,X0,t2).T

X1 = (-.1,.01)
tray1 = odeint(f,X1,t2).T

X2 = (1,.01)
tray2 = odeint(f,X2,t).T

X3 = (-.5,.01)
tray3 = odeint(f,X3,t2).T



plt.figure()
plt.plot(tray[0],tray[1])
plt.plot(tray1[0],tray1[1])
plt.plot(tray2[0],tray2[1])
plt.plot(tray3[0],tray3[1])

a = tray[:,-1]
a2 = tray3[:,-1]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [34]:
np.where(t > 20)[0][0]

2000

In [35]:
r = np.linspace(0,100,100)

In [43]:
if 2 == 1:
    print(1)
else
    print(0)

SyntaxError: invalid syntax (<ipython-input-43-2121d736330b>, line 3)

In [154]:
np.linspace(0.00000012111341,0.00000012111343,3)

array([1.2111341e-07, 1.2111342e-07, 1.2111343e-07])

In [163]:
np.linspace(0.055283522234,0.055283522267,3)

array([0.05528352, 0.05528352, 0.05528352])

In [184]:
plt.figure()
plt.plot(tray[0])

/home/lorenzo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
x

NameError: name 'x' is not defined

In [16]:
x

array([ 0.0000000000000000e+00,  2.5935508590441136e-03,
        9.6022084149289255e-03,  2.1252544184269362e-02,
        3.0648238038148035e-02,  2.7381735832724796e-02,
        6.4839877676904356e-03, -2.4592153784185500e-02,
       -4.0548846007054384e-02,  8.8205852338880120e-03,
        1.9892364157532877e-01,  6.0294283105162028e-01,
        1.2189394446457806e+00,  1.8630714744361188e+00,
        2.1241701368289401e+00,  1.5522194279698076e+00,
        7.5080680181635329e-02, -1.7669252033385054e+00,
       -3.1558615287344036e+00, -3.6182346905861436e+00,
       -3.2802194084009808e+00, -2.5958727954484413e+00,
       -1.9522785826801294e+00, -1.5000951545197772e+00,
       -1.2138171713740713e+00, -1.0204373729797229e+00,
       -8.7440478162136648e-01, -7.5981627763626602e-01,
       -6.6822842058774956e-01, -5.9160397709649970e-01,
       -5.2528085093803711e-01, -4.6623104000660281e-01,
       -4.1144403126518747e-01, -3.6072617469637441e-01,
       -3.1569153024235136e-01,